<a href="https://colab.research.google.com/github/nishant-sethi/DeepLearning.ai/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("Nishant")

Nishant


In [0]:
from IPython.display import Image

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p drive 
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers
from tqdm import tqdm
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using TensorFlow backend.


In [0]:
import os
from os import listdir
import cv2
import numpy as np
train_path='/content/drive/My Drive/images/'
def load_dataset():
    images=os.listdir(train_path)
    X=[]
    image_size=128
    for img in images:
      img=os.path.join(train_path,img)
#       print(img)
      image = cv2.imread(img)
      image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
      image = image.astype(np.float32)
      image = np.multiply(image, 1.0 / 255.0)
      X.append(image)
    X_train=np.array(X).reshape(1819,49152)
    print(X_train.shape)
    print("Data loaded")
    return X_train

In [6]:
load_dataset()

(1819, 49152)
Data loaded


array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.77647066, 0.7607844 , 0.7568628 , ..., 0.73333335, 0.7176471 ,
        0.7137255 ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.7803922 , 0.76470596, 0.7607844 , ..., 0.7490196 , 0.73333335,
        0.7294118 ],
       [0.77647066, 0.7607844 , 0.7568628 , ..., 0.7372549 , 0.72156864,
        0.7176471 ]], dtype=float32)

In [0]:
randomDim = 100
adam = Adam(lr=0.0002, beta_1=0.5)

In [0]:
def load_generator():
  generator = Sequential()
  generator.add(Dense(256, input_dim=randomDim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(512))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(1024))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(2048))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(49152, activation='tanh'))
  generator.compile(loss='binary_crossentropy', optimizer=adam)
  print("Generator is Ready")
  return generator

In [0]:
def load_discriminator():
  discriminator = Sequential()
  discriminator.add(Dense(1024, input_dim=49152, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(256))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(1, activation='sigmoid'))
  discriminator.compile(loss='binary_crossentropy', optimizer=adam)
  print("Discriminator is ready")
  return discriminator


In [0]:
def model_GAN():
  # Combined network
  discriminator.trainable = False
  ganInput = Input(shape=(randomDim,))
  x = generator(ganInput)
  ganOutput = discriminator(x)
  gan = Model(inputs=ganInput, outputs=ganOutput)
  gan.compile(loss='binary_crossentropy', optimizer=adam)
  return gan

In [0]:
epochs,batchSize=40,128
def train_model(X_train,generator,discriminator,gan):
    print("Training started")
    batchCount = X_train.shape[0] // batchSize
    print('Epochs:', epochs)
    print ('Batch size:', batchSize)
    print ('Batches per epoch:', batchCount)

    for e in range(1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(range(batchCount)):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]

            # Generate fake MNIST images
            generatedImages = generator.predict(noise)
            print(np.shape(imageBatch), np.shape(generatedImages))
            X = np.concatenate([imageBatch, generatedImages])

            # Labels for generated and real data
            yDis = np.zeros(2*batchSize)
            # One-sided label smoothing
            yDis[:batchSize] = 0.9

            # Train discriminator
            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, yDis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        # Store loss of most recent batch from this epoch
        dLosses.append(dloss)
        gLosses.append(gloss)

        if e == 1 or e % 20 == 0:
            plotGeneratedImages(e)
            saveModels(e,generator,discriminator)

    # Plot losses from every epoch
    plotLoss(e)

In [0]:
dLosses = []
gLosses = []
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('/content/drive/My Drive/images/gan_loss_epoch_%d.png' % epoch)

In [0]:
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 128, 128,3)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('/content/drive/My Drive/generated_images/gan_generated_image_epoch_%d.png' % epoch)

In [0]:
def saveModels(epoch,generator,discriminator):
    generator.save('/content/drive/My Drive/models/gan_generator_epoch_%d.h5' % epoch)
    discriminator.save('/content/drive/My Drive/models/gan_discriminator_epoch_%d.h5' % epoch)

In [26]:
if __name__ == '__main__':
    X_train=load_dataset()
    generator=load_generator()
    discriminator=load_discriminator()
    gan=model_GAN()
    train_model(X_train,generator,discriminator,gan)

error: ignored